In [1]:
import pandas as pd
import warnings
from kobert_tokenizer import KoBERTTokenizer
warnings.simplefilter(action='ignore', category=FutureWarning)

df = pd.read_csv('GKSL3k_edited.csv')
df = df.drop_duplicates()
df.drop(['video_num'], axis=1, inplace=True)
df.drop(['question_w_q_morph'], axis=1, inplace=True)
df.drop(['dataset'], axis=1, inplace=True)
df.rename(columns={'Gloss level Korean Sign Language (GKSL) sentence': 'GKSL'}, inplace=True)
df.rename(columns={'Word level Korean Language (WKL) sentence': 'WKL'}, inplace=True)


# 뼈곳, #NAME? 행 & 바꾸다1, 와이파이1 & ?, '  ' 제거
df = df[df['GKSL'].str.contains('뼈곳') == False]
df = df[df['GKSL'].str.contains('#NAME?') == False]
df['GKSL'] = df['GKSL'].str.replace('바꾸다1', '바꾸다')
df['GKSL'] = df['GKSL'].str.replace('바꾸다2', '바꾸다')
df['GKSL'] = df['GKSL'].str.replace('?', '')
df['GKSL'] = df['GKSL'].str.replace('  ', ' ')
df['GKSL'] = df['GKSL'].str.strip()
df['GKSL'] = df.apply(lambda row: row['GKSL'] + ' ?' if row['question'] else row['GKSL'], axis=1)
df['WKL'] = df['WKL'].str.replace('  ', ' ')
df['WKL'] = df['WKL'].str.strip()
df.drop(['question'], axis=1, inplace=True)
df['label'] = df['WKL'].apply(lambda x: 1 if '?' in x else 0)
df

# filtered_df = df[(df['GKSL'].str.contains('\?'))]
# filtered_df = df[(df['WKL'].str.endswith('?')) & (df['question'] == False)]
# filtered_df

/home/dodant/anaconda3/envs/kobert/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,GKSL,WKL,label
0,집 불,집에 불이 났어요.,0
1,집 도둑 넘어오다,집에 도둑이 들어왔어요.,0
2,누가 교통사고,어떤 사람이 교통사고를 당했어요.,0
3,먹다 중간 목 실수 삼키다,식사를 하다가 목에 음식이 걸렸어요.,0
4,홍수 집 물 채우다,홍수가 나서 집이 물에 잠기고 있어요.,0
...,...,...,...
3046,도착 시간 저녁 9시 정도,도착시간은 오후 9시가 되겠습니다,0
3047,아이 지금 5 되다,아이가 다섯 살이 되었다,0
3048,잘 되다 부탁,잘 되길 바래,0
3049,미래 의사 원하다,커서 의사가 되고 싶다,0


In [2]:
df.to_csv('GKSL3k_labeled.csv', index=False)

In [12]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

df = pd.read_csv('GKSL3k.csv')
df['GKSL_encoded'] = df['GKSL'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
df['WKL_encoded'] = df['WKL'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
df

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


,GKSL,WKL,GKSL_encoded,WKL_encoded
0,집 불,집에 불이 났어요.,"[2, 4384, 2485, 3]","[2, 4384, 6896, 2485, 7096, 517, 5671, 6857, 5..."
1,집 도둑 넘어오다,집에 도둑이 들어왔어요.,"[2, 4384, 517, 5859, 5908, 1465, 6964, 5782, 3]","[2, 4384, 6896, 517, 5859, 5908, 7096, 1804, 6..."
2,누가 교통사고,어떤 사람이 교통사고를 당했어요.,"[2, 1527, 1113, 3]","[2, 3224, 2589, 1113, 6116, 1618, 7874, 54, 3]"
3,먹다 중간 목 실수 삼키다,식사를 하다가 목에 음식이 걸렸어요.,"[2, 2010, 5782, 4258, 2068, 3042, 2650, 7573, ...","[2, 3008, 6116, 4932, 5330, 2068, 6896, 3609, ..."
4,홍수 집 물 채우다,홍수가 나서 집이 물에 잠기고 있어요.,"[2, 5108, 6629, 4384, 2135, 4451, 7005, 5782, 3]","[2, 5108, 6630, 1381, 4384, 7096, 2135, 6896, ..."
...,...,...,...,...
3035,도착 시간 저녁 9시 정도,도착시간은 오후 9시가 되겠습니다,"[2, 1723, 2962, 3991, 627, 6705, 4099, 3]","[2, 1723, 6706, 7086, 3434, 627, 6705, 5330, 1..."
3036,아이 지금 5 되다,아이가 다섯 살이 되었다,"[2, 3121, 4299, 611, 1763, 5782, 3]","[2, 3121, 5330, 1571, 2643, 7096, 517, 5896, 3]"
3037,잘 되다 부탁,잘 되길 바래,"[2, 3942, 1763, 5782, 2458, 3]","[2, 3942, 1763, 5585, 2186, 6023, 3]"
3038,미래 의사 원하다,커서 의사가 되고 싶다,"[2, 2155, 3629, 3533, 7798, 3]","[2, 4652, 6553, 3629, 5330, 1764, 3072, 3]"


In [13]:
df.to_csv('GKSL3k_final.csv', index=False)

In [14]:
df = pd.read_csv('GKSL13k_edited.csv')
df = df.drop_duplicates()
df.drop(['idx'], axis=1, inplace=True)
df.drop(['augmentation_method'], axis=1, inplace=True)
df.rename(columns={'Gloss level Korean Sign Language (GKSL) sentence': 'GKSL'}, inplace=True)
df.rename(columns={'Word level Korean Language (WKL) sentence': 'WKL'}, inplace=True)

df = df[df['GKSL'].str.contains('동작') == False]
df = df[df['GKSL'].str.contains('대학교') == False]
df = df[df['WKL'].str.contains('대학교') == False]
df['GKSL'] = df['GKSL'].str.replace('  ', ' ')
df['GKSL'] = df['GKSL'].str.replace('뼈곳', '곳')
df['GKSL'] = df['GKSL'].str.replace('와이파이1', '와이파이')
df['GKSL'] = df['GKSL'].str.replace('와이파이2', '와이파이')
df['GKSL'] = df['GKSL'].str.replace('바꾸다1', '바꾸다')
df['GKSL'] = df['GKSL'].str.replace('바꾸다2', '바꾸다')
df['GKSL'] = df['GKSL'].str.replace('T 동아리', '팀')
df['GKSL'] = df['GKSL'].str.replace('T 승용차', '택시')
df['GKSL'] = df['GKSL'].str.replace('ㅈ 오늘', '지금')
df['GKSL'] = df['GKSL'].str.replace('ㅈ 세계', '지구')
df['GKSL'] = df['GKSL'].str.replace('ㅈ 두다', '주차')
df['GKSL'] = df['GKSL'].str.replace('광 주', '광주')
df['GKSL'] = df['GKSL'].str.replace('?', '')
df['GKSL'] = df.apply(lambda row: row['GKSL'] + ' ?' if '?' in row['WKL'] else row['GKSL'], axis=1)
df['GKSL'] = df['GKSL'].str.strip()
df['WKL'] = df['WKL'].str.replace('tax refund ', '')
df['WKL'] = df['WKL'].str.replace('  ', ' ')
df['WKL'] = df['WKL'].str.strip()
df

,GKSL,WKL
0,집 불,댁에 불이 났어요.
1,집 불,주택에 불이 났어요.
2,집 불,가옥에 불이 났어요.
3,집 불,세대에 불이 났어요.
4,도착 10분 전,10분 내로 당도합니다.
...,...,...
13042,배우다 곳 다니다,학원에 다니다
13043,캐리어 차 어디 곳 ?,캐리어 차를 재는 곳은 어디인가요?
13044,중 사이 곳 내리다 맞다,중간정류장에서 내려주세요.
13045,무덤 장소 곳 가다,묘지로 가주세요.


In [15]:
df['GKSL_encoded'] = df['GKSL'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
df['WKL_encoded'] = df['WKL'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
df.to_csv('GKSL13k_final.csv', index=False)

In [23]:
df = pd.read_csv('GKSL13k(edited).csv')
df = df.drop_duplicates()
df.drop(['idx'], axis=1, inplace=True)
df.drop(['augmentation_method'], axis=1, inplace=True)
df.rename(columns={'Gloss level Korean Sign Language (GKSL) sentence': 'GKSL'}, inplace=True)
df.rename(columns={'Word level Korean Language (WKL) sentence': 'WKL'}, inplace=True)
filtered_df = df[df['WKL'].str.endswith('운')]
filtered_df

,GKSL,WKL
1565,쉽다,쉬운
2186,춥다,추운


In [7]:
import pandas as pd

def append_ending(row):
    if row['gloss'][-1] not in '.?!':
        row['gloss'] += ' ' + row['spoken'][-1]
    return row

def label_based_on_ending(spoken):
    if spoken[-1] == '.':
        return 0
    elif spoken[-1] == '?':
        return 1
    elif spoken[-1] == '!':
        return 2
    else:
        return None

df = pd.read_csv('GKSL13k(final).csv')
df['spoken'] = df['spoken'].apply(lambda x: x if x[-1] in '.?!' else x + '.')
df = df.apply(append_ending, axis=1)
df['label'] = df['spoken'].apply(label_based_on_ending)
df.to_csv('GKSL13k(final)_.csv', index=False)

In [11]:
df = pd.read_csv('../MY_DATA/mydata(edited).csv')
longest_string_row = df['spoken'].str.len().idxmax()
longest_string = df.loc[longest_string_row, 'spoken']
print(longest_string, len(longest_string))

df = pd.read_csv('GKSL13k(final)_.csv')
longest_string_row = df['spoken'].str.len().idxmax()
longest_string = df.loc[longest_string_row, 'spoken']
print(longest_string, len(longest_string))

df = pd.read_csv('GKSL3k(final)_.csv')
longest_string_row = df['spoken'].str.len().idxmax()
longest_string = df.loc[longest_string_row, 'spoken']
print(longest_string, len(longest_string))

나는 어린 아이들에게 도덕적 가치를 가르치는 프로그램을 개발했다. 36
나는 산에서 야생 동물을 만났습니다. 그들이 나를 공격할 것이라고 생각합니다. 43
여기 서대문구 홍제동 한양아파트인데 옥상에서 사람이 떨어지려고해요. 37


In [2]:
import pandas as pd

df = pd.read_csv('GKSL3k(final)_.csv')
df = df.sample(frac=1).reset_index(drop=True)
df.to_csv('GKSL3k(final)_mix.csv', index=False)

In [17]:
import pandas as pd

df = pd.read_csv('GKSL13k.csv')
df.drop(['augmentation_method'], axis=1, inplace=True)
df.drop(['idx'], axis=1, inplace=True)
df.rename(columns={'Gloss level Korean Sign Language (GKSL) sentence': 'gloss'}, inplace=True)
df.rename(columns={'Word level Korean Language (WKL) sentence': 'spoken'}, inplace=True)
df['gloss'] = df['gloss'].str.replace('  ', ' ')
df['combined'] = df['gloss'] + df['spoken']
df = df[df['combined'].str.len() < 32]
df.to_csv('GKSL13k_train.csv')
df

,gloss,spoken,combined
0,집 불,댁에 불이 났어요.,집 불댁에 불이 났어요.
1,집 불,주택에 불이 났어요.,집 불주택에 불이 났어요.
2,집 불,가옥에 불이 났어요.,집 불가옥에 불이 났어요.
3,집 불,세대에 불이 났어요.,집 불세대에 불이 났어요.
4,도착 10분 전,10분 내로 당도합니다.,도착 10분 전10분 내로 당도합니다.
...,...,...,...
13042,배우다 곳 다니다,학원에 다니다,배우다 곳 다니다학원에 다니다
13043,캐리어 차 어디 곳,캐리어 차를 재는 곳은 어디인가요?,캐리어 차 어디 곳캐리어 차를 재는 곳은 어디인가요?
13044,중 사이 곳 내리다 맞다,중간정류장에서 내려주세요.,중 사이 곳 내리다 맞다중간정류장에서 내려주세요.
13045,무덤 장소 곳 가다,묘지로 가주세요.,무덤 장소 곳 가다묘지로 가주세요.


In [18]:
import pandas as pd

df = pd.read_csv('GKSL3k(final).csv')
df['gloss'] = df['gloss'].str.replace('  ', ' ')
df['combined'] = df['gloss'] + df['spoken']
df = df[df['combined'].str.len() < 32]
df.to_csv('GKSL3k_train.csv')
df

,gloss,spoken,combined
0,집 불,집에 불이 났어요.,집 불집에 불이 났어요.
1,집 도둑 넘어오다,집에 도둑이 들어왔어요.,집 도둑 넘어오다집에 도둑이 들어왔어요.
2,누가 교통사고,어떤 사람이 교통사고를 당했어요.,누가 교통사고어떤 사람이 교통사고를 당했어요.
4,홍수 집 물 채우다,홍수가 나서 집이 물에 잠기고 있어요.,홍수 집 물 채우다홍수가 나서 집이 물에 잠기고 있어요.
6,누가 자살 중,어떤 사람이 자살을 시도하고 있어요.,누가 자살 중어떤 사람이 자살을 시도하고 있어요.
...,...,...,...
3033,친하다 애인 가능,친한 친구는 애인이 될 수도 있다,친하다 애인 가능친한 친구는 애인이 될 수도 있다
3034,되다,되다,되다되다
3036,아이 지금 5 되다,아이가 다섯 살이 되었다,아이 지금 5 되다아이가 다섯 살이 되었다
3037,잘 되다 부탁,잘 되길 바래,잘 되다 부탁잘 되길 바래


In [8]:
import pandas as pd

df = pd.read_csv('../MY_DATA/shots/gloss2text/0-shot.csv')
df = df.sample(frac=1).reset_index(drop=True)
train_df = df[:int(len(df) * 0.8)]
test_df = df[int(len(df) * 0.8):]
train_df.to_csv('../MY_DATA/split_train.csv', index=False)
test_df.to_csv('../MY_DATA/split_test.csv', index=False)

In [4]:
gksl3k = 'GKSL3k.csv'
gksl13k = 'GKSL13k.csv'
df1 = pd.read_csv(gksl3k)
df2 = pd.read_csv(gksl13k)
gloss_combined = pd.concat([df1['gloss'], df2['gloss']], ignore_index=True)
spoken_combined = pd.concat([df1['spoken'], df2['spoken']], ignore_index=True)

df = pd.DataFrame({
    'gloss': gloss_combined,
    'spoken': spoken_combined
})

df = df.sample(frac=1).reset_index(drop=True)
train_df = df[:int(len(df) * 0.8)]
test_df = df[int(len(df) * 0.8):]
train_df.to_csv('GKSL_split_train.csv', index=False)
test_df.to_csv('GKSL_split_test.csv', index=False)